In [ ]:
# file_path= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\Upsampling\\input\\'
# target_path= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\Upsampling\\output\\'

In [ ]:
file_path='G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\Edgebox\\Versuchsdaten\\5s_data\\corrected\\'
target_path= 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\Edgebox\\Versuchsdaten\\5s_data\\upsampling\\'

In [ ]:
target_sampling_rate= 5
upsampling_columns= 'welle_z'
upsampled_variable= 'welle_z_iterpolated'

In [ ]:
# def create_block(start_date:datetime, period: int, count_insertions: int):
#     insertion_block= pd.DataFrame({'date': [ datetime.strptime(start_date,"%Y-%m-%d %H:%M:%S") + timedelta(seconds= i*period) for i in range(1,count_insertions + 1)],'t_bett': [np.NaN for i in range(count_insertions)],'t_motor':[np.NaN for i in range(count_insertions)], 't_spindel': [np.NaN for i in range(count_insertions)],'z_welle_ok':[np.NaN for i in range(count_insertions)]})
#     return insertion_block

In [ ]:
# def insert_block(original_data: pd.DataFrame, period:int,  count_insertions: int):
#     output_df= None
#     for index, row in original_data.iterrows():
#         if index + 1 == len(original_data):
#             output_df= pd.concat([output_df,original_data[index]])
#         else:
#             start_date= row['dat_uhz']
#             block= create_block(start_date= start_date, period= period, count_insertions= count_insertions)
#             if index == 0:
#                 output_df= pd.concat([ original_data[index], block])
#             else:
#                 output_df= pd.concat([output_df, original_data[index], block])
#     return output_df


Read file

In [ ]:
# os.chdir(file_path)
# for file in glob.glob('*.csv'):
#     df= pd.read_csv(file)
#     df.reset_index(inplace= True)
#     print(file)
#     to_up_sampled_data= df[upsampling_columns] 
#     to_up_sampled_data= insert_block(to_up_sampled_data, period= 5, count_insertions= 24)
#     to_up_sampled_data['M8']=0
#     to_up_sampled_data['M121']=0
#     to_up_sampled_data['M127']=0
#     to_up_sampled_data['M7']=0
#     #to_up_sampled_data.interpolate(method='cubicspline', order=3, axis= 0,inplace= True)
#     #to_up_sampled_data.head(30)

Interpolate 5 seconds data

In [ ]:
os.chdir(file_path)
df= None
for file in glob.glob('*.csv'):
    df= pd.read_csv(file)
    print(file)
    df[upsampled_variable]= df[upsampling_columns]
    current_welle_z= 1000000
    for index, row in df.iterrows():
        if row[upsampled_variable]!= current_welle_z: ### A New value is available
            current_welle_z= row[upsampled_variable]
        else:
            df.at[index,upsampled_variable]= np.NaN
    df[upsampled_variable].interpolate(method='linear', order=3, axis= 0,inplace= True)
    # for i in range(len(df)-1,-1,-1):
    #     if df.iloc[i]['given2model']:
    #         df= df[:i+1]
    #         break
    #print(df.head(3))
    df.to_csv(target_path+'interpolated_'+file)
    scatter_mode= 'lines'
    visualization_columns= ['t_bett','t_motor','t_spindle',upsampling_columns]
    y_axis_names= visualization_columns
    fig= make_subplots(rows=len(visualization_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= visualization_columns, vertical_spacing=0.02)
    for i in range(len(visualization_columns)):
        fig.add_trace(go.Scatter(x=df['date'] ,y= df[visualization_columns[i]], name=str(visualization_columns[i]), mode= scatter_mode), row= i+1, col= 1)
        fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
    fig.add_trace(go.Scatter(x=df['date'] ,y= df[upsampled_variable], name=upsampled_variable, mode= scatter_mode), row= len(visualization_columns), col= 1)
    fig.update_layout(height=1800, width=1200, title_text="5 Sekunden-Daten")
    fig.show()
    

In [ ]:
# scatter_mode= 'lines'
# visualization_columns= ['t_bett','t_motor','t_spindle','welle_z','spline_interp']
# df= pd.read_csv(filepath_or_buffer=file_path)
# df['welle_z']= df['welle_z'].shift(-5)
# y_axis_names= visualization_columns# ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
# fig= make_subplots(rows=len(visualization_columns)-1,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= visualization_columns, vertical_spacing=0.02)
# for i in range(len(visualization_columns)):
#     if i == len(visualization_columns) -1:
#         fig.add_trace(go.Scatter(x= df['date'],y= df[visualization_columns[i]], name=visualization_columns[i], mode= scatter_mode), row= i, col= 1)
#         fig.update_yaxes(title_text= y_axis_names[i], row= i, col= 1)
#     else:
#         fig.add_trace(go.Scatter(x= df['date'],y= df[visualization_columns[i]], name=visualization_columns[i], mode= scatter_mode), row= i+1, col= 1)
#         fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
# #fig.add_trace(go.Scatter(x= df['date'],y= predictions / 1000, name='Model prediction', mode= scatter_mode), row= len(visualization_columns) -1, col= 1) 
# fig.update_layout(height=1800, width=1200, title_text="5 Sekunden-Daten")
# fig.show()


In [ ]:
# to_up_sampled_data= df[['t_bett','t_motor','given2model','t_spindle','welle_z']]
# to_up_sampled_data.loc[to_up_sampled_data['given2model'] == False, ['t_motor','t_spindle','welle_z']]= to_up_sampled_data['t_motor'].values[0] + np.NaN, to_up_sampled_data['t_spindle'].values[0] + np.NaN, to_up_sampled_data['welle_z'].values[0] + np.NaN
# to_up_sampled_data.head(10)

In [ ]:
# to_up_sampled_data.interpolate(method='cubicspline', order=3, axis= 0,inplace= True)

In [ ]:
# scatter_mode= 'lines'
# visualization_columns= ['t_bett','t_motor','t_spindle','welle_z']
# two_min_values= df[df['given2model']== True]
# y_axis_names= visualization_columns# ['ENC1_POS|8 (degee)     ', 'CMD_SPEED|8 (degree/s)    ','is_speed (degree/s)   ', 'is_acceleration(degree/s^2)   ',  'POWER|8 (W)',  'CURRENT|8 (A)',  'ENC2_POS|3 (mm)','t_spindle', 'DRZ', 't_motor','supply_temp', 't_bett', 'distance']
# fig= make_subplots(rows=len(visualization_columns),cols=1,shared_xaxes= True, print_grid= True, subplot_titles= visualization_columns, vertical_spacing=0.02)
# for i in range(len(visualization_columns)):
#     fig.add_trace(go.Scatter(x= df['date'],y= to_up_sampled_data[visualization_columns[i]], name= str(visualization_columns[i] + ' upsampled'), mode= scatter_mode), row= i+1, col= 1)
#     fig.add_trace(go.Scatter(x= df['date'], y= df[visualization_columns[i]], name=str(visualization_columns[i] + ' orginal'), mode= scatter_mode), row= i+1, col= 1)
#     fig.add_trace(go.Scatter(x=two_min_values['date'] ,y= two_min_values[visualization_columns[i]], name=str(visualization_columns[i] + '2_Min_orginal'), mode= 'markers'), row= i+1, col= 1)
#     fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
# #fig.add_trace(go.Scatter(x= df['date'],y= predictions / 1000, name='Model prediction', mode= scatter_mode), row= len(visualization_columns) -1, col= 1) 
# fig.update_layout(height=1800, width=1200, title_text="5 Sekunden-Daten")
# fig.show()